In [4]:
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
import seaborn as sns
import plotly.express as px

In [24]:
def url_get_contents(url):
 
    # Opens a website and read its
    # binary contents (HTTP Response Body)
 
    #making request to the website
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = urllib.request.Request(url=url, headers=hdr)
    f = urllib.request.urlopen(req)
 
    #reading contents of the website
    return f.read()

In [249]:
link = 'https://en.wikipedia.org/wiki/2020%E2%80%9321_Arsenal_F.C._season'
xhtml = url_get_contents(link).decode('utf-8')

In [250]:
p = HTMLTableParser()
p.feed(xhtml)

In [251]:
len(p.tables)

91

In [253]:
df = pd.DataFrame(p.tables[(64-38-11):(64-11)])
df[0]

0     [12 September 2020  1, Fulham, 0–3, Arsenal, F...
1     [19 September 2020  2, Arsenal, 2–1, West Ham ...
2     [28 September 2020  3, Liverpool, 3–1, Arsenal...
3     [4 October 2020  4, Arsenal, 2–1, Sheffield Un...
4     [17 October 2020  5, Manchester City, 1–0, Ars...
5     [25 October 2020  6, Arsenal, 0–1, Leicester C...
6     [1 November 2020  7, Manchester United, 0–1, A...
7     [8 November 2020  8, Arsenal, 0–3, Aston Villa...
8     [22 November 2020  9, Leeds United, 0–0, Arsen...
9     [29 November 2020  10, Arsenal, 1–2, Wolverham...
10    [6 December 2020  11, Tottenham Hotspur, 2–0, ...
11    [13 December 2020  12, Arsenal, 0–1, Burnley, ...
12    [16 December 2020  13, Arsenal, 1–1, Southampt...
13    [19 December 2020  14, Everton, 2–1, Arsenal, ...
14    [26 December 2020  15, Arsenal, 3–1, Chelsea, ...
15    [29 December 2020  16, Brighton Hove Albion, 0...
16    [2 January 2021  17, West Bromwich Albion, 0–4...
17    [14 January 2021  18, Arsenal, 0–0, Crysta

In [254]:
def pro_df(df, team):
    df['date'] = [x[0] for x in df[0]]
    df['home'] = [x[1].lower() for x in df[0]]
    df['score'] = [x[2] for x in df[0]]
    df['away'] = [x[3].lower() for x in df[0]]
    df['referee'] = [(' ').join(x[4].split(' ')[-2:]) for x in df[1]]
    df['points'] = 0
    
    for i in range(0, df.shape[0]):
        
        [x1, x2] = df['score'].iloc[i].split('–') 
        [x1, x2] = [int(x1), int(x2)]
        
        if (team == df['home'].iloc[i] and x1 > x2) or (team == df['away'].iloc[i] and x2 > x1):
            df['points'].iloc[i] = 3
        elif x1 == x2:
            df['points'].iloc[i] = 1
        else:
            df['points'].iloc[i] = 0
         
    
    return df

In [255]:
df = pro_df(df, "arsenal")

In [256]:
df_main = df_main.append(df)
df_use = df_main[['date','home','score','away','points','referee']]

In [264]:
df_use.index = list(range(0, df_use.shape[0]))

In [273]:
df_use['date_col'] = [pd.to_datetime((' ').join(x.split(' ')[0:3])) for x in df_use['date']]

<ipython-input-273-8fe7fe7eecee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['date_col'] = [pd.to_datetime((' ').join(x.split(' ')[0:3])) for x in df_use['date']]


In [278]:
def home_adv(x, team):
    if x == team:
        return 1
    else:
        return 0

In [282]:
df_use['home-adv'] = 0
df_use['home-adv'] = df_use['home'].apply(lambda x: home_adv(x, 'arsenal'))

<ipython-input-282-f61d5d46c05f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['home-adv'] = 0
<ipython-input-282-f61d5d46c05f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['home-adv'] = df_use['home'].apply(lambda x: home_adv(x, 'arsenal'))


In [284]:
df_final = df_use.copy()

In [317]:
df_final = df_final[['date_col', 'home', 'away', 'score', 'points', 'home-adv', 'referee']]

In [309]:
def get_GD(df):
    
    for i in range(0, df.shape[0]):
        [x1, x2] = df['score'].iloc[i].split('–') 
        [x1, x2] = [int(x1), int(x2)]
        
        if df['points'].iloc[i] == 3:
            df['GD'].iloc[i] = 

C:\Users\vishn\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [318]:
df_final.to_csv('prem-arsenal.csv', index=None)

## Plotting

In [3]:
df_ar = pd.read_csv('prem-arsenal.csv')

In [24]:
sns.set(rc={'figure.figsize':(25, 15)})

In [26]:
# sns.countplot(df_ar['referee'])

In [16]:
fig = px.pie(df_ar, values='points', names='referee', title='Points earned vs Referees', hover_data=['referee'])
fig.write_html("f1.html")

In [18]:
fig = px.pie(df_ar, values='points', names='home-adv', title="Points earned vs home-advantage")
fig.write_html("f2.html")

In [28]:
fig = px.histogram(df_ar, x='referee')
fig.write_html('f3.html')